In [ ]:
# load libraries
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, log_loss
from scipy import stats

In [ ]:
# specify file paths
features = "peakmat_AAC.csv"
response = "pac_AAC.csv"

# read in files
X = pd.read_csv(features)
X = X.iloc[:, 1:]                       # remove cell line labels
y = pd.read_csv(response)['response']   # read only drug response column

In [ ]:
# split the training dataframe into train and val
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Un-penalized Logistic Model Training and Testing

In [ ]:
# initialize linear regression model
reg = linear_model.LinearRegression()

# fit model
reg.fit(X_train, y_train)

# get predicted values for test data
y_pred = reg.predict(X_test)

# compute correlations
s_cor = stats.spearmanr(y_pred, y_test)
p_cor = stats.pearsonr(y_pred, y_test)
print('Spearman correlation:', s_cor[0])
print('Pearson correlation:', p_cor[0])

Spearman correlation: 0.6121212121212121
Spearman correlation: 0.5992668824291314


LASSO Model Training and Testing

In [ ]:
# initialize LASSO model
lasso = linear_model.Lasso()

# specify parameters for optimization
parameters = {
    'alpha': [0.1, 1, 10, 100],
    'max_iter': [500, 1000, 5000, 7500]
  }

# identify optimal parameters
reg = GridSearchCV(
    estimator = lasso,
    param_grid = parameters,
    #verbose=2
  )
reg.fit(X_train, y_train)
print('Best params:', reg.best_params_ )

Best params: {'alpha': 0.1, 'max_iter': 500}


In [ ]:
# test best model parameters on test data
reg_best = reg.best_estimator_

# get predicted values for test data
y_pred = reg_best.predict(X_test)

# compute correlations
s_cor = stats.spearmanr(y_pred, y_test)
p_cor = stats.pearsonr(y_pred, y_test)
print('Spearman correlation:', s_cor[0])
print('Pearson correlation:', p_cor[0])

    chr1.1260419.1260919  chr1.2206130.2206630  chr1.2371989.2372489  \
27                     0                     0                     0   
39                     0                     0                     0   
26                     1                     1                     1   
43                     0                     0                     0   
24                     0                     0                     0   
36                     1                     0                     0   
12                     0                     0                     0   
19                     0                     0                     0   
4                      0                     1                     0   
25                     0                     0                     0   

    chr1.3431680.3432180  chr1.4397996.4398496  chr1.5784840.5785340  \
27                     0                     0                     0   
39                     0                     0                 

<ipython-input-28-ca6d8b1d7989>:11: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  s_cor = stats.spearmanr(y_pred, y_test)
<ipython-input-28-ca6d8b1d7989>:12: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  p_cor = stats.pearsonr(y_pred, y_test)


Ridge Model Training and Testing

In [ ]:
# initialize Ridge model
ridge = linear_model.Ridge()

# fit model
ridge.fit(X_train, y_train)

# get predicted values for test data
y_pred = reg.predict(X_test)

# compute correlations
s_cor = stats.spearmanr(y_pred, y_test)
p_cor = stats.pearsonr(y_pred, y_test)
print('Spearman correlation:', s_cor[0])
print('Pearson correlation:', p_cor[0])

Spearman correlation: nan
Pearson correlation: nan


<ipython-input-29-1ba05d665f6a>:11: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  s_cor = stats.spearmanr(y_pred, y_test)
<ipython-input-29-1ba05d665f6a>:12: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  p_cor = stats.pearsonr(y_pred, y_test)


ElasticNet Model Training and Testing

In [ ]:
# initialize Elastic Net model
en = linear_model.ElasticNet()

# specify parameters for optimization
parameters = {
    'alpha': [0.1, 1, 10, 100],
    'l1_ratio': [0.2, 0.5, 0.8],
    'max_iter': [1000, 5000, 7500]
  }

# identify optimal parameters
reg = GridSearchCV(
    estimator = en,
    param_grid = parameters,
    #verbose=2
  )
reg.fit(X_train, y_train)
print('Best params:', reg.best_params_ )

In [ ]:
# test best model parameters on test data
reg_best = reg.best_estimator_

# get predicted values for test data
y_pred = reg_best.predict(X_test)

# compute correlations
s_cor = stats.spearmanr(y_pred, y_test)
p_cor = stats.pearsonr(y_pred, y_test)
print('Spearman correlation:', s_cor[0])
print('Pearson correlation:', p_cor[0])

In [ ]:
### Investigate feature importance

# initialize and fit Elastic Net model
en = linear_model.ElasticNet(alpha = 0.1, l1_ratio = 0.2, max_iter = 1000)
en.fit(X_train, y_train)

# get coefficients
coefficients = en.coef_

# create feature importance dataframe
feature_importance = pd.DataFrame({
    'Peak': X_train.columns,
    'Weight': coefficients
}).sort_values(by='Weight', ascending=False)

feature_importance.to_csv('feature_importance.csv', index=False)
feature_importance

,Peak,Weight
251,chr3.40649300.40649800,0.036250
471,chr7.134751917.134752417,0.018981
128,chr1.216815488.216815988,0.015451
421,chr6.134908052.134908552,0.011477
543,chr9.13718685.13719185,0.000352
...,...,...
735,chr12.178091.178591,-0.008403
793,chr14.37598144.37598644,-0.010007
873,chr16.88905594.88906094,-0.017537
381,chr5.174724906.174725406,-0.023270


Random Forest Training and Testing

In [ ]:
# initialize Random Forest model
rf = RandomForestRegressor()

# specify parameters for optimization
parameters = {
    'n_estimators': [10, 50, 100, 150, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2, 5],
    'max_features': ['sqrt', 'log2', 50, 100, 200]
}

# identify optimal parameters
reg = GridSearchCV(
    estimator = rf,
    param_grid = parameters,
    #verbose=2
  )
reg.fit(X_train, y_train)
print('Best params:', reg.best_params_ )

Best params: {'max_depth': None, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 150}


/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


In [ ]:
# test best model parameters on test data
reg_best = reg.best_estimator_

# get predicted values for test data
y_pred = reg_best.predict(X_test)

# compute correlations
s_cor = stats.spearmanr(y_pred, y_test)
p_cor = stats.pearsonr(y_pred, y_test)
print('Spearman correlation:', s_cor[0])
print('Pearson correlation:', p_cor[0])

Spearman correlation: 0.6606060606060605
Pearson correlation: 0.6139311308747906
